# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#your code here
spaceship.shape

(8693, 14)

In [4]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [6]:
spaceship.isnull().sum()

spaceship.dropna(inplace=True)

In [7]:
#your code here
spaceship['Cabin'].value_counts()


# Función para mantener solo la primera letra si está en el conjunto {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}
def filtrar_primera_letra(valor):
    if valor[0] in {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}:
        return valor[0]
    return None

# Aplicar la función a la columna
spaceship['Cabin'] = spaceship['Cabin'].apply(filtrar_primera_letra)

spaceship['Cabin'].value_counts()

Cabin
F    2152
G    1973
E     683
B     628
C     587
D     374
A     207
T       2
Name: count, dtype: int64

In [8]:
spaceship = spaceship.drop(['PassengerId', 'Name'], axis=1)

In [9]:
non_numeric_columns = spaceship.select_dtypes(include=['object']).columns

df_dummies = pd.get_dummies(spaceship, columns=non_numeric_columns)

In [10]:
df_dummies.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,False,True,False,...,False,False,False,False,False,False,False,True,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,True,False,False,...,False,False,True,False,False,False,False,True,True,False


**Perform Train Test Split**

In [22]:
#your code here

from sklearn.preprocessing import StandardScaler


X = df_dummies.drop('Transported', axis=1)

y = df_dummies['Transported']

# División en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Crear el escalador y ajustarlo solo en el conjunto de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Transformar el conjunto de prueba usando el escalador ajustado
X_test_scaled = scaler.transform(X_test)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [27]:
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Supongamos que ya tienes X_train, X_test, y_train, y_test

# Bagging
bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=1.0,
    bootstrap=True,  # Con reemplazo
    n_jobs=-1
)
bagging_clf.fit(X_train_scaled, y_train)
y_pred_bagging = bagging_clf.predict(X_test_scaled)
print("Bagging accuracy:", accuracy_score(y_test, y_pred_bagging))

# Pasting
pasting_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=1.0,
    bootstrap=False,  # Sin reemplazo
    n_jobs=-1
)
pasting_clf.fit(X_train_scaled, y_train)
y_pred_pasting = pasting_clf.predict(X_test_scaled)
print("Pasting accuracy:", accuracy_score(y_test, y_pred_pasting))

Bagging accuracy: 0.8048411497730711
Pasting accuracy: 0.7579425113464447


- Random Forests

In [28]:
#your code here

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Supongamos que ya tienes X_train, X_test, y_train, y_test

# Random Forest
rf_clf = RandomForestClassifier(
    n_estimators=100,  # Número de árboles en el bosque
    max_features='sqrt',  # Número de características a considerar en cada división
    n_jobs=-1,  # Usar todos los núcleos disponibles
    random_state=42  # Para reproducibilidad
)
rf_clf.fit(X_train_scaled, y_train)
y_pred_rf = rf_clf.predict(X_test_scaled)
print("Random Forest accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest accuracy: 0.8101361573373677


- Gradient Boosting

In [30]:
#your code here

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Supongamos que ya tienes X_train, X_test, y_train, y_test

# Gradient Boosting
gb_clf = GradientBoostingClassifier(
    n_estimators=100,  # Número de árboles en el ensamble
    learning_rate=0.1,  # Tasa de aprendizaje
    max_depth=3,  # Profundidad máxima de los árboles individuales
    random_state=42  # Para reproducibilidad
)
gb_clf.fit(X_train_scaled, y_train)
y_pred_gb = gb_clf.predict(X_test_scaled)
print("Gradient Boosting accuracy:", accuracy_score(y_test, y_pred_gb))

Gradient Boosting accuracy: 0.8101361573373677


- Adaptive Boosting

In [31]:
#your code here

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Supongamos que ya tienes X_train, X_test, y_train, y_test

# Adaptive Boosting (AdaBoost)
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),  # Estimador base
    n_estimators=100,  # Número de estimadores
    learning_rate=1.0,  # Tasa de aprendizaje
    random_state=42  # Para reproducibilidad
)
ada_clf.fit(X_train_scaled, y_train)
y_pred_ada = ada_clf.predict(X_test_scaled)
print("AdaBoost accuracy:", accuracy_score(y_test, y_pred_ada))

AdaBoost accuracy: 0.7745839636913767


Which model is the best and why?

In [32]:
#comment here

#Random forest y Gradient son los mejores modelos, ya que tienen las mejores precisiones.